In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [83]:
import os
import sys
import argparse
import numpy as np
import random
import time
import matplotlib.pyplot as plt
import cv2
import h5py
from tqdm import tqdm
try:
    import queue
except ImportError:
    import Queue as queue

In [84]:
sys.path.append("../../simulator/carla/PythonAPI/carla/dist/carla-0.9.6-py3.5-linux-x86_64.egg")

In [85]:
import carla

In [86]:
def compute_data_buffer(image):
    array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
    array = np.reshape(array, (image.height, image.width, 4))
    array = array[:, :, :3]
    return array

class CarlaSyncMode(object):
    """
    Context manager to synchronize output from different sensors. Synchronous
    mode is enabled as long as we are inside this context

        with CarlaSyncMode(world, sensors) as sync_mode:
            while True:
                data = sync_mode.tick(timeout=1.0)

    """

    def __init__(self, world, *sensors, **kwargs):
        self.world = world
        self.sensors = sensors
        self.frame = None
        self.delta_seconds = 1.0 / kwargs.get('fps', 20)
        self._queues = []
        self._settings = None

    def __enter__(self):
        print("CarlaSyncMode.enter")
        self._settings = self.world.get_settings()
        self.frame = self.world.apply_settings(carla.WorldSettings(
            no_rendering_mode=False,
            synchronous_mode=True,
            fixed_delta_seconds=self.delta_seconds))

        def make_queue(register_event):
            q = queue.Queue()
            register_event(q.put)
            self._queues.append(q)

        make_queue(self.world.on_tick)
        for sensor in self.sensors:
            make_queue(sensor.listen)
        return self

    def tick(self, timeout):
        
        self.frame = self.world.tick()
        data = [self._retrieve_data(q, timeout) for q in self._queues]
        assert all(x.frame == self.frame for x in data)
        return data

    def __exit__(self, *args, **kwargs):
        print("CarlaSyncMode.exit")
        self.world.apply_settings(self._settings)

    def _retrieve_data(self, sensor_queue, timeout):
        while True:
            data = sensor_queue.get(timeout=timeout)
            if data.frame == self.frame:
                return data

In [87]:
class ClientSideBoundingBoxes(object):
    """
    This is a module responsible for creating 3D bounding boxes and drawing them
    client-side on pygame surface.
    """

    @staticmethod
    def get_bounding_boxes(vehicles, camera):
        """
        Creates 3D bounding boxes based on carla vehicle list and camera.
        """

        bounding_boxes = [ClientSideBoundingBoxes.get_bounding_box(vehicle, camera) for vehicle in vehicles]
        # filter objects behind camera
        bounding_boxes = [bb for bb in bounding_boxes if all(bb[:, 2] > 0)]
        return bounding_boxes

    @staticmethod
    def draw_bounding_boxes(image, bounding_boxes):
        """
        Draws bounding boxes on pygame display.
        """

        for bbox in bounding_boxes:

            points = [(int(bbox[i, 0]), int(bbox[i, 1])) for i in range(8)]
            # draw lines
            # base
            image = cv2.line(image, points[0], points[1], (0, 255, 0), 1)
            image = cv2.line(image, points[1], points[2], (0, 255, 0), 1)
            image = cv2.line(image, points[2], points[3], (0, 255, 0), 1)
            image = cv2.line(image, points[3], points[0], (0, 255, 0), 1)
            # top
            image = cv2.line(image, points[4], points[5], (0, 255, 0), 1)
            image = cv2.line(image, points[5], points[6], (0, 255, 0), 1)
            image = cv2.line(image, points[6], points[7], (0, 255, 0), 1)
            image = cv2.line(image, points[7], points[4], (0, 255, 0), 1)
            
            # base-top
            image = cv2.line(image, points[0], points[4], (0, 255, 0), 1)
            image = cv2.line(image, points[1], points[5], (0, 255, 0), 1)
            image = cv2.line(image, points[2], points[6], (0, 255, 0), 1)
            image = cv2.line(image, points[3], points[7], (0, 255, 0), 1)
        
        return image

    @staticmethod
    def get_bounding_box(vehicle, camera):
        """
        Returns 3D bounding box for a vehicle based on camera view.
        """

        bb_cords = ClientSideBoundingBoxes._create_bb_points(vehicle)
        cords_x_y_z = ClientSideBoundingBoxes._vehicle_to_sensor(bb_cords, vehicle, camera)[:3, :]
        cords_y_minus_z_x = np.concatenate([cords_x_y_z[1, :], -cords_x_y_z[2, :], cords_x_y_z[0, :]])
        bbox = np.transpose(np.dot(camera.calibration, cords_y_minus_z_x))
        camera_bbox = np.concatenate([bbox[:, 0] / bbox[:, 2], bbox[:, 1] / bbox[:, 2], bbox[:, 2]], axis=1)
        return camera_bbox

    @staticmethod
    def _create_bb_points(vehicle):
        """
        Returns 3D bounding box for a vehicle.
        """

        cords = np.zeros((8, 4))
        extent = vehicle.bounding_box.extent
        cords[0, :] = np.array([extent.x, extent.y, -extent.z, 1])
        cords[1, :] = np.array([-extent.x, extent.y, -extent.z, 1])
        cords[2, :] = np.array([-extent.x, -extent.y, -extent.z, 1])
        cords[3, :] = np.array([extent.x, -extent.y, -extent.z, 1])
        cords[4, :] = np.array([extent.x, extent.y, extent.z, 1])
        cords[5, :] = np.array([-extent.x, extent.y, extent.z, 1])
        cords[6, :] = np.array([-extent.x, -extent.y, extent.z, 1])
        cords[7, :] = np.array([extent.x, -extent.y, extent.z, 1])
        return cords

    @staticmethod
    def _vehicle_to_sensor(cords, vehicle, sensor):
        """
        Transforms coordinates of a vehicle bounding box to sensor.
        """

        world_cord = ClientSideBoundingBoxes._vehicle_to_world(cords, vehicle)
        sensor_cord = ClientSideBoundingBoxes._world_to_sensor(world_cord, sensor)
        return sensor_cord

    @staticmethod
    def _vehicle_to_world(cords, vehicle):
        """
        Transforms coordinates of a vehicle bounding box to world.
        """

        bb_transform = carla.Transform(vehicle.bounding_box.location)
        bb_vehicle_matrix = ClientSideBoundingBoxes.get_matrix(bb_transform)
        vehicle_world_matrix = ClientSideBoundingBoxes.get_matrix(vehicle.get_transform())
        bb_world_matrix = np.dot(vehicle_world_matrix, bb_vehicle_matrix)
        world_cords = np.dot(bb_world_matrix, np.transpose(cords))
        return world_cords

    @staticmethod
    def _world_to_sensor(cords, sensor):
        """
        Transforms world coordinates to sensor.
        """

        sensor_world_matrix = ClientSideBoundingBoxes.get_matrix(sensor.get_transform())
        world_sensor_matrix = np.linalg.inv(sensor_world_matrix)
        sensor_cords = np.dot(world_sensor_matrix, cords)
        return sensor_cords

    @staticmethod
    def get_matrix(transform):
        """
        Creates matrix from carla transform.
        """

        rotation = transform.rotation
        location = transform.location
        c_y = np.cos(np.radians(rotation.yaw))
        s_y = np.sin(np.radians(rotation.yaw))
        c_r = np.cos(np.radians(rotation.roll))
        s_r = np.sin(np.radians(rotation.roll))
        c_p = np.cos(np.radians(rotation.pitch))
        s_p = np.sin(np.radians(rotation.pitch))
        matrix = np.matrix(np.identity(4))
        matrix[0, 3] = location.x
        matrix[1, 3] = location.y
        matrix[2, 3] = location.z
        matrix[0, 0] = c_p * c_y
        matrix[0, 1] = c_y * s_p * s_r - s_y * c_r
        matrix[0, 2] = -c_y * s_p * c_r - s_y * s_r
        matrix[1, 0] = s_y * c_p
        matrix[1, 1] = s_y * s_p * s_r + c_y * c_r
        matrix[1, 2] = -s_y * s_p * c_r + c_y * s_r
        matrix[2, 0] = s_p
        matrix[2, 1] = -c_p * s_r
        matrix[2, 2] = c_p * c_r
        return matrix

In [116]:
class DataStorageWriter:

    def __init__(self, file_path, sensor_names=["lidar1", "camera_rgb1", "camera_semseg1", "gnss", "bouding_boxes"]):
        self.h5 = h5py.File(file_path, 'w')
        self.groups = dict()
        for group_name in sensor_names:
            self.groups[group_name] = self.h5.create_group(group_name)
    
    def write_lidar(self, sensor_name, index, lidar_measurement):
        x, y, z = list(), list(), list()
        for location in lidar_measurement:
            x.append(location.x)
            y.append(location.y)
            z.append(location.z)
        value = np.hstack([np.array(x).reshape((-1, 1)), np.array(y).reshape((-1, 1)), np.array(z).reshape((-1, 1))])
        self.write_matrix(sensor_name, index, value)
    
    def write_gnss(self, sensor_name, index, gnss_data):
        x, y, z = list(), list(), list()
        x.append(gnss_data.latitude)
        y.append(gnss_data.longitude)
        z.append(gnss_data.altitude)
        value = np.hstack([np.array(x).reshape((-1, 1)), np.array(y).reshape((-1, 1)), np.array(z).reshape((-1, 1))])
        self.write_matrix(sensor_name, index, value)
        
    def write_image(self, sensor_name, index, image):
        self.write_matrix(sensor_name, index, image)

    def write_matrix(self, sensor_name, index, value):
        self.groups[sensor_name].create_dataset(str(index), data=value, maxshape=value.shape, chunks=True)

    def close(self):
        self.h5.close()
        self.h5 = None


In [89]:
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

In [103]:
world = client.get_world()

In [104]:
weather = world.get_weather()
weather.cloudyness = 0
weather.precipitation = 0
weather.precipitation_deposits = 0
weather.wind_intensity = 0
weather.sun_azimuth_angle = 30
weather.sun_altitude_angle = 100
world.set_weather(weather)

In [117]:
blueprint_library = world.get_blueprint_library()
actor_list = []

Add EGO Vehicle

In [118]:
bp = random.choice(blueprint_library.filter('vehicle.tesla.*'))
transform = carla.Transform(carla.Location(x=-88.5, y=-160, z=0.8), carla.Rotation(pitch=0.000000, yaw=90.0, roll=0.000000))
ego_vehicle = world.spawn_actor(bp, transform)
actor_list.append(ego_vehicle)

Add Other Opponents (Vehicles)

In [119]:
bp = random.choice(blueprint_library.filter('vehicle.tesla.*'))
transform = carla.Transform(carla.Location(x=-88.5, y=-120, z=0.8), carla.Rotation(pitch=0.000000, yaw=90.0, roll=0.000000))
op_vehicle1 = world.spawn_actor(bp, transform)
actor_list.append(op_vehicle1)

Create Sensors

In [120]:
VIEW_WIDTH = 1024//2
VIEW_HEIGHT = 768//2
VIEW_FOV = 90

camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', str(VIEW_WIDTH))
camera_bp.set_attribute('image_size_y', str(VIEW_HEIGHT))
camera_bp.set_attribute('fov', str(VIEW_FOV))

camera_rgb = world.spawn_actor(
    camera_bp,
    carla.Transform(carla.Location(x=-5.5, z=2.8), carla.Rotation(pitch=-25)),
    attach_to=ego_vehicle)

calibration = np.identity(3)
calibration[0, 2] = VIEW_WIDTH / 2.0
calibration[1, 2] = VIEW_HEIGHT / 2.0
calibration[0, 0] = calibration[1, 1] = VIEW_WIDTH / (2.0 * np.tan(VIEW_FOV * np.pi / 360.0))

camera_rgb.calibration = calibration

actor_list.append(camera_rgb)

In [121]:
camera_semseg_bp = blueprint_library.find('sensor.camera.semantic_segmentation')
camera_semseg_bp.set_attribute('image_size_x', str(VIEW_WIDTH))
camera_semseg_bp.set_attribute('image_size_y', str(VIEW_HEIGHT))
camera_semseg_bp.set_attribute('fov', str(VIEW_FOV))

camera_semseg = world.spawn_actor(
    camera_semseg_bp,
    carla.Transform(carla.Location(x=-5.5, z=2.8), carla.Rotation(pitch=-25)),
    attach_to=ego_vehicle)

camera_semseg.calibration = calibration

actor_list.append(camera_semseg)

In [122]:
camera_depth_bp = blueprint_library.find('sensor.camera.depth')
camera_depth_bp.set_attribute('image_size_x', str(VIEW_WIDTH))
camera_depth_bp.set_attribute('image_size_y', str(VIEW_HEIGHT))
camera_depth_bp.set_attribute('fov', str(VIEW_FOV))

camera_depth = world.spawn_actor(
    camera_depth_bp,
    carla.Transform(carla.Location(x=-5.5, z=2.8), carla.Rotation(pitch=-25)),
    attach_to=ego_vehicle)

camera_depth.calibration = calibration

actor_list.append(camera_depth)

In [123]:
lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')

lidar = world.spawn_actor(
    lidar_bp,
    carla.Transform(carla.Location(x=1.5, z=2.4)),
    attach_to=ego_vehicle
)

In [124]:
gnss_bp = blueprint_library.find('sensor.other.gnss')
gnss_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
gnss = world.spawn_actor(
    gnss_bp,
    gnss_transform,
    attach_to=ego_vehicle
)

In [125]:
vehicles = world.get_actors().filter('vehicle.*')

In [127]:
try:
    ds = DataStorageWriter("../../dataset/data3.h5", ["lidar", "camera", "gnss", "semantic"])        
    with CarlaSyncMode(world, camera_rgb, camera_semseg, lidar, gnss, fps=30) as sync_mode:
        ego_vehicle.set_velocity(carla.Vector3D(0.0, 10.3, 0))
        op_vehicle1.set_velocity(carla.Vector3D(0.0, 5.3, 0))
        for idx in range(30*10):
            snapshot, image_rgb, image_seg, lidar_pcl, gnss_data = sync_mode.tick(timeout=2.0)
            image_seg.convert(carla.ColorConverter.CityScapesPalette)
            np_image = compute_data_buffer(image_rgb)

            bounding_boxes = ClientSideBoundingBoxes.get_bounding_boxes(vehicles, camera_rgb)
            np_image2 = ClientSideBoundingBoxes.draw_bounding_boxes(np_image, bounding_boxes)
            frame = snapshot.frame
            ts = int(snapshot.timestamp.elapsed_seconds * 1e6)
            
            ds.write_lidar("lidar", ts, lidar_pcl)
            ds.write_image("camera", ts,  compute_data_buffer(image_rgb))
            ds.write_image("semantic", ts,  compute_data_buffer(image_seg))
            ds.write_gnss("gnss", ts, gnss_data)
            
            cv2.imshow("img", np_image2)
            
            if cv2.waitKey(1) == ord('q'):
                break
            if ego_vehicle.get_location().y > 140:
                break
finally:
    for actor in actor_list:
        actor.destroy()
    actor_list = []
    cv2.destroyAllWindows()
    ds.close()

OSError: Unable to create file (unable to truncate a file which is already open)